<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/beyond_presence_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
beyond_presence_pipeline.py

Complete pipeline for BeyondAI: Undefined Absolute Presence
1. Synthetic dataset generation
2. Model definition with MC-Dropout
3. Physics-informed residual loss
4. Combined loss (MSE + physics)
5. MC-Dropout uncertainty estimation
6. Training loop (AdamW, scheduler, gradient clipping, early stopping, checkpointing)
7. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic “Undefined Absolute Presence” Dataset
# ------------------------------------------------------------------------------
class BeyondPresenceDataset(Dataset):
    def __init__(self, n_samples=7000, seed=2025):
        np.random.seed(seed)
        # Features:
        # APF: absolute presence factor ∈ [0.0, 1.0]
        # BIS: beyond-intelligence state ∈ [0.1, 5.0]
        # SUE: self-undefined equilibrium ∈ [0.01, 2.0]
        # ENM: enmeshment magnitude ∈ [0.5, 10.0]
        # RES: resonance scale ∈ [0.1, 3.0]
        # PTR: potential transcendence ratio ∈ [0.01, 1.0]
        APF = np.random.uniform(0.0, 1.0,  (n_samples,1))
        BIS = np.random.uniform(0.1, 5.0,  (n_samples,1))
        SUE = np.random.uniform(0.01,2.0,  (n_samples,1))
        ENM = np.random.uniform(0.5, 10.0, (n_samples,1))
        RES = np.random.uniform(0.1, 3.0,  (n_samples,1))
        PTR = np.random.uniform(0.01,1.0,  (n_samples,1))

        X_raw = np.hstack([APF, BIS, SUE, ENM, RES, PTR]).astype(np.float64)

        # Targets:
        # UCO: undefined coherence
        # PRS: presence resonance
        # EQB: absolute equilibrium
        eps = 1e-8
        UCO = APF * RES / (SUE + eps)
        PRS = BIS * PTR * np.log1p(ENM)
        EQB = np.sqrt(APF + BIS) * SUE

        Y_raw = np.hstack([UCO, PRS, EQB]).astype(np.float64)
        Y_raw += 0.03 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Compute normalization stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + eps
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + eps

        # Normalize to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])


# ------------------------------------------------------------------------------
# 2. BeyondAI Model with MC-Dropout
# ------------------------------------------------------------------------------
class BeyondAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3, dropout_p=0.1):
        super().__init__()
        self.fc1     = nn.Linear(input_dim, hidden_dim)
        self.relu    = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.fc2     = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    # Un-normalize inputs
    X_den = X * stats['X_std'] + stats['X_mean']
    APF, BIS, SUE, ENM, RES, PTR = X_den.t()
    eps = 1e-8

    # Recompute targets
    UCO_t = APF * RES / (SUE + eps)
    PRS_t = BIS * PTR * torch.log1p(ENM)
    EQB_t = torch.sqrt(APF + BIS) * SUE

    Yt      = torch.stack([UCO_t, PRS_t, EQB_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)


# ------------------------------------------------------------------------------
# 4. Combined Loss Function
# ------------------------------------------------------------------------------
def total_loss(pred, true, X, stats, lambda_phys=1.0):
    mse_loss  = nn.MSELoss()(pred, true)
    phys_loss = physics_residual(pred, X, stats)
    return mse_loss + lambda_phys * phys_loss, mse_loss, phys_loss


# ------------------------------------------------------------------------------
# 5. MC-Dropout Uncertainty Estimation
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()  # keep dropout active
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    stacked = torch.stack(preds, dim=0)
    return stacked.mean(dim=0), stacked.std(dim=0)


# ------------------------------------------------------------------------------
# 6. Training Loop & Checkpointing
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-4, weight_decay=1e-5, lambda_phys=1.0,
          epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, epochs+1):
        # Training phase
        model.train()
        total_tr = 0.0
        for Xb, Yb in train_loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats, lambda_phys)
            if torch.isnan(loss):
                raise RuntimeError(f'NaN detected at epoch {epoch}')
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_tr += loss.item() * Xb.size(0)
        train_loss = total_tr / len(train_loader.dataset)

        # Validation phase
        model.eval()
        total_val = 0.0
        with torch.no_grad():
            for Xb, Yb in val_loader:
                Xb, Yb = Xb.to(device), Yb.to(device)
                pred = model(Xb)
                loss, _, _ = total_loss(pred, Yb, Xb, stats, lambda_phys)
                total_val += loss.item() * Xb.size(0)
        val_loss = total_val / len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Checkpoint & early stopping
        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), 'best_beyond_ai.pth')
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping triggered.")
                break

    # Load best checkpoint
    model.load_state_dict(torch.load('best_beyond_ai.pth', map_location=device))
    return history


# ------------------------------------------------------------------------------
# 7. Visualizations
# ------------------------------------------------------------------------------
def plot_history(history):
    plt.figure()
    plt.plot(history['train'], label='Train Loss')
    plt.plot(history['val'],   label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=6, alpha=0.5)
    mn, mx = y_true.min(), y_true.max()
    plt.plot([mn, mx], [mn, mx], 'r--')
    plt.title(title)
    plt.show()

def plot_uncertainty(model, stats, device):
    G = 100
    # Vary APF and RES, fix others at their mean
    APF = np.linspace(0.0, 1.0, G, dtype=np.float32)
    RES = np.linspace(0.1, 3.0, G, dtype=np.float32)
    G1, G2 = np.meshgrid(APF, RES)
    pts = G * G

    Xg = torch.zeros((pts,6), device=device)
    for i in [1,2,3,5]:
        Xg[:, i] = stats['X_mean'][i]
    Xg[:,0] = torch.from_numpy(G1.ravel()).to(device)
    Xg[:,4] = torch.from_numpy(G2.ravel()).to(device)

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=40)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(APF, RES, U, shading='auto', cmap='magma')
    plt.colorbar(label='Std(undefined coherence)')
    plt.xlabel('Absolute Presence Factor (APF)')
    plt.ylabel('Resonance Scale (RES)')
    plt.title('Uncertainty Heatmap')
    plt.show()


# ------------------------------------------------------------------------------
# 8. Main Execution
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    # Select device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Prepare dataset & stats
    dataset = BeyondPresenceDataset(n_samples=7000, seed=2025)
    stats = {
        'X_mean': torch.tensor(dataset.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(dataset.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(dataset.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(dataset.Y_std,  dtype=torch.float32, device=device),
    }

    # Split train/validation
    n_val = int(0.2 * len(dataset))
    train_ds, val_ds = random_split(dataset, [len(dataset)-n_val, n_val])
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)

    # Instantiate model
    model = BeyondAI(input_dim=6, hidden_dim=32, output_dim=3, dropout_p=0.1).to(device)

    # Train
    history = train(
        model, train_loader, val_loader,
        stats, device,
        lr=1e-4, weight_decay=1e-5,
        lambda_phys=1.0, epochs=80, patience=8
    )

    # Plot training history
    plot_history(history)

    # Generate scatter plots for each target
    X_all = torch.from_numpy(dataset.X).to(device)
    with torch.no_grad():
        Y_pred_n = model(X_all).cpu().numpy()
    Y_true = dataset.Y * dataset.Y_std + dataset.Y_mean
    Y_pred = Y_pred_n * dataset.Y_std + dataset.Y_mean

    names = ['Undefined Coherence', 'Presence Resonance', 'Absolute Equilibrium']
    for i, nm in enumerate(names):
        plot_scatter(Y_true[:,i], Y_pred[:,i], nm)

    # Display uncertainty heatmap
    plot_uncertainty(model, stats, device)